In [ ]:
from keras.datasets import imdb
import pandas as pd
import numpy as np
from keras.layers import LSTM, Activation, Dropout, Dense, Input, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from keras.layers import Embedding
from keras.models import Model
import string
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.model_selection import train_test_split
from keras import Sequential

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk import word_tokenize, pos_tag
import os, re, csv, math, codecs
from sklearn import model_selection
from sklearn import metrics
import torch
import torch.nn as nn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 

In [ ]:
import spacy
import re
from spacy.lang.hi import STOP_WORDS as STOP_WORDS_HI

In [ ]:
#Hindi text
with codecs.open("../input/sentiment-analysis/data/emoji_prediction_data/hindi_data/hindi.txt", encoding='utf-8') as f:
    hindi_text = f.read()#to read the total file
print(hindi_text[0:1000])

In [ ]:
sentiment_hindi = pd.read_csv('../input/sentiment-analysis/data/emoji_prediction_data/hindi_data/hindi.csv', header=None)
sentiment_hindi = sentiment_hindi[0].tolist() 

In [ ]:
#HINDI
#postive=2, neutral=0, negative=1
for i in range(0,len(sentiment_hindi)):
    if(sentiment_hindi[i]==19 or sentiment_hindi[i]==18 or sentiment_hindi[i]==17 or sentiment_hindi[i]==13 or sentiment_hindi[i]==12 or sentiment_hindi[i]==11 or sentiment_hindi[i]==9 or sentiment_hindi[i]==8):
        sentiment_hindi[i]="neutral"
    elif(sentiment_hindi[i]==15 or sentiment_hindi[i]==16):
        sentiment_hindi[i]="negative"
    else:
        sentiment_hindi[i]="positive"

In [ ]:
#hindi
def remove_punct_hi(text):
    new_words = " "
    for word in text:
        if word not in STOP_WORDS_HI:
            w = re.sub(r'[.#-:?!\_|""'',-...A-Za-z0-9]','',word) #remove everything except words and space#how 
            new_words = new_words + w
    return re.sub(r"\s+", " ", new_words, flags=re.UNICODE)

In [ ]:
hindi_text = hindi_text.splitlines()
#print(remove_punct(text_cleaned[0:10000]))
filtered_sentence_hi = []
for i in range(0,len(hindi_text)):
    filtered_sentence_hi.append(remove_punct_hi(hindi_text[i]))
print(filtered_sentence_hi[0:10])

In [ ]:
# Importing LabelEncoder from Sklearn 
# library from preprocessing Module.
from sklearn.preprocessing import LabelEncoder
 
# Creating a instance of label Encoder.
le = LabelEncoder()
 
# Using .fit_transform function to fit label
# encoder and return encoded label
label = le.fit_transform(sentiment_hindi[0:len(filtered_sentence_hi)])

label_inverse = le.inverse_transform(label)
set(label_inverse)

In [ ]:
X_train=filtered_sentence_hi
Y_train=label

In [ ]:
df = pd.DataFrame({'Hindi_text':X_train,'Sentiment':Y_train})
df.head()

In [ ]:
df['Sentiment'].value_counts()

In [ ]:
# n = 40000
# msk = df.groupby('Sentiment')['Sentiment'].transform('size') >= n
# df = pd.concat((df[msk].groupby('Sentiment').sample(n=n), df[~msk]), ignore_index=True)

In [ ]:
df['Sentiment'].value_counts()

In [ ]:
!pip install googletrans==3.1.0a0

In [ ]:
from googletrans import Translator

# Initialize the translator
translator = Translator()

In [ ]:
df = df.dropna()

In [ ]:
df['Sentiment'].value_counts()

In [ ]:
 df['English_text'] = df['Hindi_text'].apply(lambda text: translator.translate(text, src='hi', dest='en').text)

In [ ]:
X = df.drop(["Sentiment"], axis = 1)

In [ ]:
X_train, X_test,Y_train, Y_test = train_test_split(X, df['Sentiment'], stratify=df['Sentiment'], test_size=0.25, random_state = 45)

In [ ]:
Y_train.value_counts()

In [ ]:
X = list(df['Hindi_text'].values)

In [ ]:
hindi_tokenizer = Tokenizer()

all_sentences = X_train + X_test

hindi_tokenizer.fit_on_texts(X['Hindi_text'].values)

# + 1 for unknown token
vocabSize = len(hindi_tokenizer.word_index) +1

X_train_seq = hindi_tokenizer.texts_to_sequences(X_train['Hindi_text'])
X_test_seq = hindi_tokenizer.texts_to_sequences(X_test['Hindi_text'])
# X_val_seq = hindi_tokenizer.texts_to_sequences(X_val['Text'])
# Initializing max length of sentence to 20 words
max_length = 20

In [ ]:
X_train_seq_pad = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_test_seq_pad = pad_sequences(X_test_seq, maxlen=max_length,padding='post')

In [ ]:
!wget https://storage.googleapis.com/ai4bharat-public-indic-nlp-corpora/embedding/indicnlp.v1.hi.vec.gz
!gzip -d /kaggle/working/indicnlp.v1.hi.vec.gz

In [ ]:
vocabSize

In [ ]:
# Read GloVE embeddings
path_to_glove_file = '/kaggle/working/indicnlp.v1.hi.vec'
num_tokens = vocabSize
embedding_dim = 300 #latent factors or features  
hits = 0
misses = 0
embeddings_index = {}

In [ ]:
# Load GloVe word embeddings from the specified file
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs
print("Found %s word vectors." % len(embeddings_index))

# Initialize an embedding matrix for our neural network
embedding_matrix = np.zeros((num_tokens, embedding_dim))

# Assign pre-trained word vectors to our vocabulary
for word, i in hindi_tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words found in the embedding index are assigned their respective vectors
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        # Words not found in the embedding index are set to all-zeros
        # This includes the representation for "padding" and "OOV" (Out of Vocabulary)
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

In [ ]:
emb_dims = 256

model = Sequential()
model.add(Embedding(vocabSize, 300, input_length=max_length, weights=[embedding_matrix],trainable=False))
model.add(LSTM(units = 256,return_sequences=True))
model.add(LSTM(units = 128, dropout = 0.5,recurrent_dropout = 0.5,return_sequences=True))
model.add(LSTM(units = 64, dropout = 0.1,recurrent_dropout = 0.1,return_sequences=True))
model.add(LSTM(units = 16))
model.add(Dense(3, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

In [ ]:
history = model.fit(X_train_seq_pad, Y_train, epochs = 20, batch_size = 256, validation_data=(X_test_seq_pad, Y_test), shuffle=False)

In [ ]:
X_train[:5]

In [ ]:
Y_train[:5]

In [ ]:
 df_train['Text_en'] = df_train['Text'].apply(lambda text: translator.translate(text, src='hi', dest='en').text)

In [ ]:
df_train.head(10)

In [ ]:
df_train.shape

In [ ]:
df_train = df_train[df_train['Text'] != df_train['Text_en']]

In [ ]:
df_train.shape

In [ ]:
df_train.head()

In [ ]:
df_train['label'] = label

In [ ]:
df_train.head()